In [ ]:
# this assignment is for finding the arabic restaurants in the cities of Sydney

In [140]:
import numpy as np #numpy

import pandas as pd #pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests #request

from pandas.io.json import json_normalize # tranform JSON file into dataframs

import folium #love folium

print('Libraries imported.')

Libraries imported.


In [141]:
# @hidden_cell
myclientid = 'T1YS3NIS0L5IWOQG4XONX5ZGGJZC5D55NAZAQPGMWONCBFT1'
mysecretid = 'ECHTEUCNETIRTNUZRDVDMNJSCNTXVPDJQ5X5D3IHVEMXW35I'

In [142]:
CLIENT_ID = myclientid # your Foursquare ID
CLIENT_SECRET = mysecretid # your Foursquare Secret
VERSION = '20200505' # Foursquare API version

print('FourSquare Details:')
print('Client_ID: ' + CLIENT_ID + 'I have hidden my secret id using #@hidden_cell')
print('Version: ' + VERSION)

FourSquare Details:
Client_ID: T1YS3NIS0L5IWOQG4XONX5ZGGJZC5D55NAZAQPGMWONCBFT1I have hidden my secret id using #@hidden_cell
Version: 20200505


In [143]:
# my categories are middle easter restaurants, meditarien, kebab
# 4bf58dd8d48988d115941735, 4bf58dd8d48988d1c0941735, 5283c7b4e4b094cb91ec88d7

LIMIT = 100 # Maximum is 100 doesnt matter really
cities = ["Sydney, NSW, Australia", 'Melbourne, VIC, Australia', 'Adelaide, SA, Australia', 'Canberra, ACT, Australia', 'Perth, WA, Australia']
results = {}
categories = ["4bf58dd8d48988d115941735", "4bf58dd8d48988d1c0941735", "5283c7b4e4b094cb91ec88d7"]

# for category in categories:
for city in cities:
    for category in categories:   
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            city,
            LIMIT,
            category) # This ID IS FOR MIDDLE EASTERN RESTAURANTS ,Mediterian, Kebab
        results[city,category] = requests.get(url).json()

In [144]:
df_venues={}
df_venues1 = {}
frames = {}
frames1 = {}

for city in cities:
    for category in categories:

        venues = json_normalize(results[city,category]['response']['groups'][0]['items'])
        df_venues[city,category] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
        df_venues[city,category].columns = ['Name', 'Address', 'Lat', 'Lng']


#For combining the dictionary so it has only 5 key names which are cities    
for city in cities:
    
    frames[city] = [df_venues[city,categories[0]], df_venues[city,categories[1]], df_venues[city,categories[2]]]
    df_venues1[city] = pd.concat(frames[city])
    
# now we use the df_venues1 which is a dictionary that has the keys of the names of our cities
#and the values are the dataframes of all the venues under that city

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  if __name__ == '__main__':


In [166]:
maps = {}

for city in cities:
    #now we find the cities mean latitude and longitude, which for us could be a good approximate of the center
    city_latitude = df_venues1[city]['Lat'].mean(axis=0)
    city_longitude = df_venues1[city]['Lng'].mean(axis=0)
    
    maps[city] = folium.Map(location=[city_latitude, city_longitude], zoom_start=10)

        # add a marker as a restaurant in the maps
    for lat, lng, label in zip(df_venues1[city]['Lat'], df_venues1[city]['Lng'], df_venues1[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius = 3,
            popup = label,
            color='#a742f5',
            fill=True,
            fill_color='#a742f5',
            fill_opacity=0.6,
            parse_html=False).add_to(maps[city])
    print(f"Total number of all restaurants within out categories in {city} = ", len(df_venues1[city]))
    print("Showing all restaurants in the maps")

Total number of all restaurants within out categories in Sydney, NSW, Australia =  214
Showing all restaurants in the maps
Total number of all restaurants within out categories in Melbourne, VIC, Australia =  121
Showing all restaurants in the maps
Total number of all restaurants within out categories in Adelaide, SA, Australia =  32
Showing all restaurants in the maps
Total number of all restaurants within out categories in Canberra, ACT, Australia =  26
Showing all restaurants in the maps
Total number of all restaurants within out categories in Perth, WA, Australia =  57
Showing all restaurants in the maps


In [167]:
maps[cities[0]]

In [153]:
maps[cities[1]]

In [154]:
maps[cities[2]]

In [155]:
maps[cities[3]]

In [156]:
maps[cities[4]]

### We can see that Only Melbourne, Sydney and perth have a decent amount of Middle eastern restaurants. at 214 for sydney and 121 for melbourne and 57 in Perth and we are only going to account for these because they have the highest number of restaurants. So we will disregard, Canberra and Adelaide.

### For this I will use some basic statistics. I will get the mean location of the pizza places which should be near to most of them if they are really dense or far if not. 

### Next I will take the average of the distance of the venues to the mean coordinates. 

In [160]:
maps = {}
for city in cities:
    city_latitude = df_venues1[city]['Lat'].mean(axis=0)
    city_longitude = df_venues1[city]['Lng'].mean(axis=0)
    
    maps[city] = folium.Map(location=[city_latitude, city_longitude], zoom_start=10)
    
    venues_mean_coor = [df_venues1[city]['Lat'].mean(), df_venues1[city]['Lng'].mean()] 
    
    
    for lat, lng, label in zip(df_venues1[city]['Lat'], df_venues1[city]['Lng'], df_venues1[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=3,
            popup=label,
            color='#a742f5',
            fill=True,
            fill_color='#a742f5',
            fill_opacity=0.6,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="blue", weight=1.0, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=6,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])

    print(city)
    print("Mean Distance from Mean coordinates")
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues1[city][['Lat','Lng']].values)))

Sydney, NSW, Australia
Mean Distance from Mean coordinates
0.11356511981114739
Melbourne, VIC, Australia
Mean Distance from Mean coordinates
0.03625154274065976
Adelaide, SA, Australia
Mean Distance from Mean coordinates
0.04356485899492711
Canberra, ACT, Australia
Mean Distance from Mean coordinates
0.07245730562157879
Perth, WA, Australia
Mean Distance from Mean coordinates
0.041156309893264506


In [161]:
maps[cities[0]] #Sydney

In [162]:
maps[cities[1]] # Melbourne

In [163]:
maps[cities[4]] #perth

### We now see that Perth should be disregarded because it has way less restaurants than melbourne but still a higher mean average distance.
### but we cant disregard sydney because it has almost double the restaurants than melbourne and tho the mean distance is still unreliable to account for.


So what we need to do now is to reduce the number of restuarants in sydney and see to be able to compare it with melbourne
sydney has 214 restaurants while melbourne has 112 so if we disregard the furthest 102 restaurants we might have a better comparison

In [164]:
city = 'Sydney, NSW, Australia'
venues_mean_coor = [df_venues1[city]['Lat'].mean(), df_venues1[city]['Lng'].mean()] 

print(city)
print("Mean Distance from Mean coordinates")
dists = np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues1[city][['Lat','Lng']].values)
dists.sort()
print(np.mean(dists[:-103])) # Ignore the furthest 102 restaurantsdistance

Sydney, NSW, Australia
Mean Distance from Mean coordinates
0.07116531880653676


### So even with ignoring the furthest 102 restaurants in sydney the mean distance to the center of the city is still higher 
### this means melbourne is the most conjested city with delicious arabic food

Gotta start focusing my work applications to melbourne then, thank you data science

Thanks to IBM also 